<h1>Install neccessary </h1>

In [2]:
!unzip /content/mini_training.zip

Archive:  /content/mini_training.zip
   creating: mini_training/
  inflating: mini_training/s1_d2_p002_002_data_time_series.npy  
  inflating: mini_training/s1_d2_p002_002_data_time_stamps.npy  
  inflating: mini_training/s1_d2_p002_002_label_time_series.npy  
  inflating: mini_training/s1_d2_p002_002_label_time_stamps.npy  
  inflating: mini_training/s1_d2_p002_005_data_time_series.npy  
  inflating: mini_training/s1_d2_p002_005_data_time_stamps.npy  
  inflating: mini_training/s1_d2_p002_005_label_time_series.npy  
  inflating: mini_training/s1_d2_p002_005_label_time_stamps.npy  


In [3]:
!pip install braindecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [4]:
from braindecode.models.util import models_dict

print(f'All the Braindecode models:\n{list(models_dict.keys())}')

All the Braindecode models:
['ATCNet', 'Deep4Net', 'DeepSleepNet', 'EEGConformer', 'EEGITNet', 'EEGInception', 'EEGInceptionERP', 'EEGInceptionMI', 'EEGNetv1', 'EEGNetv4', 'EEGResNet', 'HybridNet', 'ShallowFBCSPNet', 'SleepStagerBlanco2020', 'SleepStagerChambon2018', 'SleepStagerEldele2021', 'TCN', 'TIDNet', 'USleep']


In [5]:
from braindecode.models import ShallowFBCSPNet


In [6]:
print(ShallowFBCSPNet.__doc__)


Shallow ConvNet model from Schirrmeister et al 2017.

Model described in [Schirrmeister2017]_.

Parameters
----------
n_chans : int
    Number of EEG channels.
n_outputs : int
    Number of outputs of the model. This is the number of classes
    in the case of classification.
n_times : int
    Number of time samples of the input window.
n_filters_time: int
    Number of temporal filters.
filter_time_length: int
    Length of the temporal filter.
n_filters_spat: int
    Number of spatial filters.
pool_time_length: int
    Length of temporal pooling filter.
pool_time_stride: int
    Length of stride between temporal pooling filters.
final_conv_length: int | str
    Length of the final convolution layer.
    If set to "auto", length of the input signal must be specified.
conv_nonlin: callable
    Non-linear function to be used after convolution layers.
pool_mode: str
    Method to use on pooling layers. "max" or "mean".
pool_nonlin: callable
    Non-linear function to be used after poolin

In [7]:
model = ShallowFBCSPNet(
    n_chans=32,
    n_times=1000,
    n_outputs=2,
    final_conv_length='auto',
)
print(model)

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 32, 1000]             [1, 2]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 32, 1000]             [1, 32, 1000, 1]          --                        --
├─Rearrange (dimshuffle): 1-2            [1, 32, 1000, 1]          [1, 1, 1000, 32]          --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1000, 32]          [1, 40, 976, 1]           52,240                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 976, 1]           [1, 40, 976, 1]           80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 976, 1]           [1, 40, 976, 1]           --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 976, 1]           [1, 40, 61, 1]            --                        [75, 1]
├─Express

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [8]:
import mne
import numpy as np

info = mne.create_info(ch_names=['C3', 'C4', 'Cz'], sfreq=256., ch_types='eeg')
X = np.random.randn(100, 3, 1024)  # 100 epochs, 3 channels, 4 seconds (@256Hz)
epochs = mne.EpochsArray(X, info=info)
y = np.random.randint(0, 4, size=100)  # 4 classes
print(epochs)

Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
<EpochsArray |  100 events (all good), 0 – 3.99609 s, baseline off, ~2.4 MB, data loaded,
 '1': 100>


In [9]:
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier

net = EEGClassifier(
    'ShallowFBCSPNet',
    module__final_conv_length='auto',
    train_split=ValidSplit(0.2),
    # To train a neural network you need validation split, here, we use 20%.
)


In [10]:
net.fit(epochs, y)


  epoch    valid_acc    valid_loss     dur
-------  -----------  ------------  ------
      1       0.1500       26.6241  0.0958
      2       0.1500       26.6241  0.0348


/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


      3       0.1500       26.6241  0.0391
      4       0.1500       26.6241  0.0309
      5       0.1500       26.6241  0.0335
      6       0.1500       26.6241  0.0357
      7       0.1500       26.6241  0.0349
      8       0.1500       26.6241  0.0331
      9       0.1500       26.6241  0.0625
     10       0.1500       26.6241  0.0313


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 3, 1024]              [1, 4]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 3, 1024]              [1, 3, 1024, 1]           --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 3, 1024, 1]           [1, 1, 1024, 3]           --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1024, 3]           [1, 40, 1000, 1]          5,840                     --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 1000, 1]          [1, 40, 1000, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

In [11]:
print(net.module_)


Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 3, 1024]              [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 3, 1024]              [1, 3, 1024, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 3, 1024, 1]           [1, 1, 1024, 3]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1024, 3]           [1, 40, 1000, 1]          5,840                     --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1000, 1]          [1, 40, 1000, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1000, 1]          [1, 40, 1000, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1000, 1]          [1, 40, 62, 1]            --                        [75, 1]
├─Express

In [12]:
print(f'{net.module_.n_chans=}\n{net.module_.n_times=}\n{net.module_.n_outputs=}'
      f'\n{net.module_.input_window_seconds=}\n{net.module_.sfreq=}\n{net.module_.chs_info=}')

net.module_.n_chans=3
net.module_.n_times=1024
net.module_.n_outputs=4
net.module_.input_window_seconds=4.0
net.module_.sfreq=256.0
net.module_.chs_info=[{'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'C3', 'scanno': 1, 'logno': 1}, {'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'C4', 'scanno': 2, 'logno': 2}, {'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FI

<hr>

<h1>Test</h1>

In [15]:
from scipy.signal import butter, filtfilt, iirnotch, freqz
import os

In [16]:
# Band-pass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Notch filter
def notch_filter(data, cut_freq, fs, Q=30):
    nyq = 0.5 * fs
    freq = cut_freq / nyq
    b, a = iirnotch(freq, Q)
    y = filtfilt(b, a, data)
    return y

def filter03050(Ipsignal):
    sampling_rate = 250  # Hz
    # fft_signal = np.fft.fft(Ipsignal)
    # frequencies = np.fft.fftfreq(len(Ipsignal), 1/sampling_rate)
    filtered_signal = bandpass_filter(Ipsignal, 1, 30, sampling_rate)          # band-pass filter 0-30 Hz
    filtered_signal = notch_filter(filtered_signal, 50, sampling_rate)       # notch filter 50 Hz
    return filtered_signal

def unique_name(Datapath):
    all_name = os.listdir(Datapath)
    new = []
    for un in all_name:
        i = un.replace("_data_time_series.npy","").replace("_data_time_stamps.npy","").replace("_label_time_series.npy","").replace("_label_time_stamps.npy","")
        new.append(i)
        unique_name = list(set(new))
    return unique_name

def check_different_duration(label_time_stamps):
    keep_time = []
    start_time = 0
    for current_time in list(label_time_stamps):
        keep_time.append(current_time-start_time)
        start_time = current_time
    keep_time = keep_time[1:]
    avgtime = sum(keep_time)/len(keep_time)
    stat = True
    for etime in keep_time:
        if 6.8 <= etime <= 7.5:
            stat = True
        else:
            stat = False
            break
    return stat,avgtime

file_path = "/content/mini_training"
unique_name = unique_name(file_path)

data_dataset  = []
label_dataset = []
for name in unique_name:
    print(name)
    signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals[:,:8]
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):
        continue
    # print("Avg time",self.avgtime)
    if filter!=None:
        filtered_signal = []
        Dim = signals.shape[1]
        print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    sample_point = int(round(avgtime))
    datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        start_point = index * datapoint
        end_point = (index + 1) * datapoint
        if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
            data_dataset.append(signals[start_point:end_point])
        else:
            break
    label_dataset.append(label_time_series)
data_dataset = np.stack(data_dataset)

label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()
# print(data_dataset)
# print(label_dataset)
data_sliced_with_label = tuple(zip(data_dataset,label_dataset))

s1_d2_p002_002
8
s1_d2_p002_005
8


In [17]:
data_dataset.shape

(60, 1750, 8)

In [18]:
data_dataset = data_dataset.reshape(60, 8, 1750)
data_dataset.shape

(60, 8, 1750)

<hr>

<h1>Experiment</h1>

In [19]:
from braindecode.models.util import models_dict

print(f'All the Braindecode models:\n{list(models_dict.keys())}')

All the Braindecode models:
['ATCNet', 'Deep4Net', 'DeepSleepNet', 'EEGConformer', 'EEGITNet', 'EEGInception', 'EEGInceptionERP', 'EEGInceptionMI', 'EEGNetv1', 'EEGNetv4', 'EEGResNet', 'HybridNet', 'ShallowFBCSPNet', 'SleepStagerBlanco2020', 'SleepStagerChambon2018', 'SleepStagerEldele2021', 'TCN', 'TIDNet', 'USleep']


In [20]:
from braindecode.models import ShallowFBCSPNet              # Import the model from braindecode * There are many model that available

In [21]:
model = ShallowFBCSPNet(
    n_chans   = 8,
    n_times   = 1750,
    n_outputs = 3,
    final_conv_length='auto',
)
print(model)

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 8, 1750]              [1, 3]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 8, 1750]              [1, 8, 1750, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 8, 1750, 1]           [1, 1, 1750, 8]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 8]           [1, 40, 1726, 1]          13,840                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1726, 1]          [1, 40, 1726, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1726, 1]          [1, 40, 111, 1]           --                        [75, 1]
├─Express

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [27]:
import mne
import numpy as np

In [28]:
info = mne.create_info(
    ch_names = ['c1','c2','c3','c4','c5','c6','c7','c8'], sfreq=250., ch_types="eeg"
)
# info = mne.create_info(ch_names=['C3', 'C4', 'Cz'], sfreq=256., ch_types='eeg')
# X = np.random.randn(100, 3, 1024)  # 100 epochs, 3 channels, 4 seconds (@256Hz)
epochs = mne.EpochsArray(data_dataset, info=info)
# y = np.random.randint(0, 4, size=100)  # 4 classes
# print(epochs)

Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated


In [29]:
# ##########################################################
# y = np.random.randint(0, 4, size=100)  # 4 classes
# y

array([1, 3, 3, 3, 1, 2, 0, 3, 1, 1, 0, 1, 3, 3, 3, 0, 0, 3, 3, 0, 0, 0,
       1, 1, 2, 0, 3, 2, 1, 3, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 3, 2, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 3, 1, 2, 3, 1, 3, 0, 2, 3, 3, 0, 3, 2, 2,
       0, 3, 2, 0, 3, 1, 1, 3, 2, 0, 3, 2, 0, 0, 2, 3, 2, 0, 0, 1, 2, 0,
       3, 0, 1, 3, 2, 1, 1, 0, 2, 2, 2, 3])

In [30]:
label_dataset = np.array(label_dataset)
label_dataset

array([120, 110, 150, 150, 120, 110, 120, 110, 150, 150, 120, 110, 120,
       150, 110, 150, 120, 110, 120, 110, 150, 150, 120, 110, 150, 120,
       110, 150, 120, 110, 120, 110, 150, 150, 120, 110, 120, 110, 150,
       150, 120, 110, 120, 150, 110, 150, 120, 110, 120, 110, 150, 150,
       120, 110, 150, 120, 110, 150, 120, 110])

In [31]:
# Map value
mapping = {110: 0, 120: 1, 150: 2}

mapped_label = np.vectorize(mapping.get)(label_dataset)
print(mapped_label)

[1 0 2 2 1 0 1 0 2 2 1 0 1 2 0 2 1 0 1 0 2 2 1 0 2 1 0 2 1 0 1 0 2 2 1 0 1
 0 2 2 1 0 1 2 0 2 1 0 1 0 2 2 1 0 2 1 0 2 1 0]


In [32]:
label = np.array(mapped_label)
label
################################################################

array([1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 1, 2, 0, 2, 1, 0, 1, 0, 2, 2,
       1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 1, 2,
       0, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 1, 0, 2, 1, 0])

In [33]:
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier

net = EEGClassifier(
    'ShallowFBCSPNet',
    module__final_conv_length='auto',
    train_split=ValidSplit(0.2),
    # To train a neural network you need validation split, here, we use 20%.
)
# Train model
net.fit(epochs, label)


  epoch    valid_acc    valid_loss     dur
-------  -----------  ------------  ------
      1       0.5833       38.9595  0.0444
      2       0.5833       38.9595  0.0354
      3       0.5833       38.9595  0.0355
      4       0.5833       38.9595  0.0364


/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


      5       0.5833       38.9595  0.0393
      6       0.5833       38.9595  0.0371
      7       0.5833       38.9595  0.0517
      8       0.5833       38.9595  0.0388
      9       0.5833       38.9595  0.0373
     10       0.5833       38.9595  0.0353


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 8, 1750]              [1, 3]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 8, 1750]              [1, 8, 1750, 1]           --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 8, 1750, 1]           [1, 1, 1750, 8]           --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 8]           [1, 40, 1726, 1]          13,840                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

In [57]:
print(f'{net.module_.n_chans=}\n{net.module_.n_times=}\n{net.module_.n_outputs=}'
      f'\n{net.module_.input_window_seconds=}\n{net.module_.sfreq=}\n{net.module_.chs_info=}')

net.module_.n_chans=8
net.module_.n_times=1750
net.module_.n_outputs=3
net.module_.input_window_seconds=7.0
net.module_.sfreq=250.0
net.module_.chs_info=[{'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'c1', 'scanno': 1, 'logno': 1}, {'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'c2', 'scanno': 2, 'logno': 2}, {'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FI

<h1>Predict</h1>

In [34]:
! unzip /content/Test_data.zip

Archive:  /content/Test_data.zip
   creating: Test_data/
  inflating: Test_data/s1_d2_p003_001_data_time_series.npy  
  inflating: Test_data/s1_d2_p003_001_data_time_stamps.npy  
  inflating: Test_data/s1_d2_p003_001_label_time_series.npy  
  inflating: Test_data/s1_d2_p003_001_label_time_stamps.npy  


In [36]:
test_data = np.load("/content/Test_data/s1_d2_p003_001_data_time_series.npy")

In [41]:
data_dataset  = []
label_dataset = []
file_path = "/content/Test_data"
unique_name = ["s1_d2_p003_001"]
for name in unique_name:
    print(name)
    signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals[:,:8]
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):
        continue
    # print("Avg time",self.avgtime)
    if filter!=None:
        filtered_signal = []
        Dim = signals.shape[1]
        print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    sample_point = int(round(avgtime))
    datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        start_point = index * datapoint
        end_point = (index + 1) * datapoint
        if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
            data_dataset.append(signals[start_point:end_point])
        else:
            break
    label_dataset.append(label_time_series)

data_dataset = np.stack(data_dataset)

label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()

s1_d2_p003_001
8


In [46]:
data_dataset = data_dataset.reshape(30,8,1750)
data_dataset.shape

(30, 8, 1750)

In [47]:
test_epochs = mne.EpochsArray(data_dataset,info=info)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


In [48]:
# Get predictions from the trained model
predictions = net.predict(test_epochs)
probabilities = net.predict_proba(test_epochs)

In [49]:
print("Predictions:", predictions)
print("Probabilities:", probabilities)

Predictions: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Probabilities: [[2.1184293e-23 0.0000000e+00 1.0000000e+00]
 [1.4548094e-21 0.0000000e+00 1.0000000e+00]
 [7.5185719e-22 0.0000000e+00 1.0000000e+00]
 [4.6278721e-22 0.0000000e+00 1.0000000e+00]
 [4.8391015e-22 0.0000000e+00 1.0000000e+00]
 [2.9542154e-22 0.0000000e+00 1.0000000e+00]
 [2.7656325e-22 0.0000000e+00 1.0000000e+00]
 [2.1029472e-22 0.0000000e+00 1.0000000e+00]
 [9.8257573e-23 0.0000000e+00 1.0000000e+00]
 [7.2325262e-22 0.0000000e+00 1.0000000e+00]
 [1.1301319e-21 0.0000000e+00 1.0000000e+00]
 [1.0181615e-21 0.0000000e+00 1.0000000e+00]
 [5.9576482e-22 0.0000000e+00 1.0000000e+00]
 [2.3422681e-23 0.0000000e+00 1.0000000e+00]
 [3.9782401e-24 0.0000000e+00 1.0000000e+00]
 [6.1452643e-22 0.0000000e+00 1.0000000e+00]
 [2.5049063e-22 0.0000000e+00 1.0000000e+00]
 [1.5195583e-21 0.0000000e+00 1.0000000e+00]
 [1.1567777e-21 0.0000000e+00 1.0000000e+00]
 [3.8662596e-22 0.0000000e+00 1.0000000e+00]
 [1.100600